# 第6章 动态类型简介

## 缺少类型声明语句的情况

- 在Python中，类型是在运行过程中自动决定的，而不是通过代码声明

## 变量、对象和引用

- 变量创建：一个变量（也就是说变量名），就像a，当代码第一次给它赋值时就创建了它，之后的赋值将会改变已创建的变量名的值
- 变量类型： 变量永远不会有任何的和它关联的类型信息或约束，类型属于对象而不是变量，变量是通用的，它只是在特定的时间点引用了一个特定的对象
- 变量使用：当变量出现在表达式中，它会立马被当前引用的对象所代替，无论对象是什么类型，变量必须在使用前被赋值

**总而言之，变量在赋值的时候才创建，它可以引用任何类型的对象，并且必须在引用之前赋值**

举个例子，当输入`a = 3`时，从概念上来说，Python将会执行以下三个步骤：
1. 创建一个对象来代表3
2. 创建一个变量a，如果它还没有创建的话
3. 将变量a作为对象3的引用

做一些进一步的解释：
- 变量是一个系统表的元素，拥有指向对象的连接的空间
- 对象是分配的一块内存，有足够的空间去表示它们所代表的值；另，每个对象都有两个标准的头部信息：一个类型标识符去标识这个对象的类型，以及一个引用的计数器，用来决定是不是可以回收这个对象
- 引用是自动形成的从变量到对象的指针（变量中存储着对象所在的内存地址）

- 作为一种优化，Python缓存了不变的对象并对其进行复用，例如，小的整数和字符串（如，每一个‘0’都不是一块真正的、新的内存块），具体可参考[\[1\]](https://www.cnblogs.com/c-x-m/articles/7749968.html)和[\[2\]](https://blog.csdn.net/lnotime/article/details/81772992)

## 对象的垃圾收集（Garbage Collection，GC）

- 在Python中，每个对象有一个计数器，记录当前指向该对象的引用数目，一旦计数器被置为0，该对象的内存空间就会被自动回收（放入自由空间池，等待后来的对象使用）

- Python的GC，采用的是引用计数机制为主，标记-清除和分代回收两种机制为辅的策略，具体可参考[这篇博客](https://blog.csdn.net/u012294618/article/details/83181011)

## 共享引用

- 多个变量名引用了同一个对象，即为共享引用

In [1]:
a = 3
b = a# a 和 b 引用了同一个对象“3”，它们指向了相同的内存空间
print a
print b

3
3


In [2]:
a = 3
b = a
a = 'spam'# Python 创建了一个新对象“spam”，并设置a对这个新对象进行引用，b仍然引用原始的对象“3”
print a
print b

spam
3


In [3]:
a = 3
b = a
a = a + 2 # 由于整数是不可变的，因此a+2生成了一个新的对象，并设置a对这个新对象进行引用，b仍然引用原始的对象“3”
print a
print b

5
3


## 共享引用和在原处修改

- 对于支持在原处修改的对象（也即可变对象），共享引用时，对一个变量的修改会影响到其它变量

In [4]:
# L1和L2引用了同一个列表对象，这个列表对象又包括了对3个整型对象的引用；L1[0] = 100，这个语句使得列表对象的第0个元素指向了“100”这个新对象，但是列表对象本身的内存地址并没有改变；因此，L1和L2依然引用之前的列表对象，只不过列表对象所引用的元素发生了变化
L1 = [2, 3, 4]
L2 = L1
print L1
print L2
L1[0] = 100
print L1
print L2

[2, 3, 4]
[2, 3, 4]
[100, 3, 4]
[100, 3, 4]


- 如果想避免这种情况，可以拷贝对象：使用分片表达式、使用对象的copy方法（set\dict）、copy模块，除了copy模块中的deepcopy是深拷贝，其余的都是浅拷贝，即只拷贝顶层对象

In [5]:
import copy
L1 = [2, 3, [10, 100, 1000, ['1', '2']], 'a']
L2 = L1[:]
L3 = copy.copy(L1)
L4 = copy.deepcopy(L1)
print L1
print L2
print L3
print L4

[2, 3, [10, 100, 1000, ['1', '2']], 'a']
[2, 3, [10, 100, 1000, ['1', '2']], 'a']
[2, 3, [10, 100, 1000, ['1', '2']], 'a']
[2, 3, [10, 100, 1000, ['1', '2']], 'a']


In [6]:
L1.append(6)
print L1
print L2
print L3
print L4

[2, 3, [10, 100, 1000, ['1', '2']], 'a', 6]
[2, 3, [10, 100, 1000, ['1', '2']], 'a']
[2, 3, [10, 100, 1000, ['1', '2']], 'a']
[2, 3, [10, 100, 1000, ['1', '2']], 'a']


In [7]:
L1[2][0] = -10
print L1
print L2
print L3
print L4

[2, 3, [-10, 100, 1000, ['1', '2']], 'a', 6]
[2, 3, [-10, 100, 1000, ['1', '2']], 'a']
[2, 3, [-10, 100, 1000, ['1', '2']], 'a']
[2, 3, [10, 100, 1000, ['1', '2']], 'a']


In [8]:
L1[2][3][0] = 'a'
print L1
print L2
print L3
print L4

[2, 3, [-10, 100, 1000, ['a', '2']], 'a', 6]
[2, 3, [-10, 100, 1000, ['a', '2']], 'a']
[2, 3, [-10, 100, 1000, ['a', '2']], 'a']
[2, 3, [10, 100, 1000, ['1', '2']], 'a']


## 共享引用和相等

- “==”操作符，可以测试两个被引用的对象是否有相同的值；“is”操作符，可以测试两个变量名是否精确地指向同一个对象；id函数可以获取对象的内存地址，如果两个对象的内存地址一致，则它们是同一个对象，因此该函数的作用等同于is（也有特殊情况，这里不讨论）

In [9]:
L1 = [1, 2, 3]
L2 = [1, 2, 3]
print L1 == L2
print L1 is L2
print id(L1), id(L2)

True
False
4382370272 4383269232


In [10]:
L1 = [1, 2, 3]
L2 = L1
print L1 == L2
print L1 is L2
print id(L1), id(L2)

True
True
4383119048 4383119048


In [11]:
# Python 缓存了小的整数和字符串，因此a和b均指向了同一个对象
a = 3
b = 3
print a == b
print a is b
print id(a), id(b)

True
True
140654118206984 140654118206984


In [12]:
a = 1994 * 1995
b = 1994 * 1995
print a == b
print a is b
print id(a), id(b)

True
False
140654156174800 140654156174848


- 采用sys模块中的getrefcount函数可以返回对象的引用次数

In [13]:
import sys
sys.getrefcount(1)# 绝大多数都是被系统代码所引用的

2204